# Bibliotecas

In [1877]:
# ! pip install selenium -U --quiet
# ! apt-get update -U --quiet
# ! apt install chromium-chromedriver -U --quiet

In [1878]:
import  pandas as pd
import  numpy as np
import  re
import  plotly.express as px
import  plotly.graph_objects as go
import  matplotlib.pyplot as plt
import  requests

from    plotly.subplots import make_subplots
from    sklearn.decomposition import PCA
from    sklearn.preprocessing import (StandardScaler,
                                      Normalizer,
                                      LabelEncoder)
from    sklearn.model_selection import (train_test_split,
                                        KFold)
from    sklearn.metrics import (confusion_matrix,
                                accuracy_score,
                                mean_absolute_error,
                                mean_squared_error)
from    scipy.stats import bartlett
from    selenium import webdriver
from    datetime import datetime
from    google.colab import drive

In [1879]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Webdriver

In [1880]:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_experimental_option('prefs', {'intl': {'timezone': 'Sao_Paulo'}})

# Web scraping

In [1881]:
# wd = webdriver.Chrome(options=options)

In [1882]:
# ano_atual = datetime.now().year

# ano_atual

In [1883]:
# temporadas_jokic = np.arange(2015, ano_atual + 1)
# # Soma 1 para que o range inclua a temporada atual

# lista_frames_scrapping = []

# df_stats_jokic_raw = pd.DataFrame()

# # Coleta os dados do jogador nas temporadas
# for i, ano in enumerate(temporadas_jokic):

#     url = 'https://sports.yahoo.com/nba/players/5352/gamelog/?guccounter=1&guce_referrer       \
#            =aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAALAJHhWY9VoPRZ3jQV_Kza8xe    \
#            Il8O9eRZUhKOU9gZA7T3hcnrB3sxOAIQEacPNovcAf1Hf_A72rqE-qIWiZeeZHLeRQiFWpryJBxU1vH7ne  \
#            twg81HkJ6TJipGCdzUL-SUdACT_gx9sYkMYeNOPX126leoR5LSS1YTGlm1XbyCARt&selectedTable=0&season=' + str(ano)

#     # Verifica a requisição
#     response = requests.get(url)

#     if response.ok:
#         print(ano)
#         print('This URL is working properly!\n')
#     else:
#         print(ano)
#         print('This page either does not exist or is out.')

#     wd.get(url)

#     html = wd.page_source

#     frame_scrapping = pd.read_html(html)

#     lista_frames_scrapping.append([ano, frame_scrapping[0]])
#     # Add o ano da temporada no dataset
#     lista_frames_scrapping[i][1]['Temporada'] = lista_frames_scrapping[i][0]

#     # Concatena todas em temporadas em um dataset
#     df_stats_jokic_raw = pd.concat([df_stats_jokic_raw, lista_frames_scrapping[i][1]], axis=0)

In [1884]:
# df_stats_jokic_raw

In [1885]:
# df_stats_jokic_raw.columns = df_stats_jokic_raw.columns.droplevel(0)

# df_stats_jokic_raw.rename(columns={'': 'Temporada'}, inplace=True)

# df_stats_jokic_raw

In [1886]:
# df_stats_jokic_raw.isnull().sum()

In [1887]:
# df_stats_jokic_raw.dtypes

In [1888]:
# df_stats_jokic_raw.drop(columns=['+/-'], inplace=True)

In [1889]:
# df_stats_jokic_raw.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
#                           encoding='utf-8',
#                           sep=',',
#                           index=False)

# Dataset raw from csv

In [1890]:
df_stats_jokic_raw = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
                                 encoding='utf-8',
                                 sep=',')

df_stats_jokic_raw.reset_index(drop=True, inplace=True)

In [1891]:
df_stats_jokic_raw

,Date,Opp,Score,Type,Min,FGM,FGA,FG%,3PM,3PA,...,OR,DR,Reb,Ast,TO,Stl,Blk,PF,Pts,Temporada
0,Apr 14,@POR,L 99-107,Reg,34:56,7,10,70.0,1,2,...,6,7,13,5,2,0,0,3,18,2015
1,Apr 10,UTA,L 100-84,Reg,36:02,7,9,77.8,0,2,...,3,8,11,4,1,2,1,4,19,2015
2,Apr 9,SA,W 98-102,Reg,32:13,3,9,33.3,0,2,...,3,12,15,4,0,1,1,2,8,2015
3,Apr 6,OKC,L 124-102,Reg,19:03,3,5,60.0,1,3,...,0,2,2,2,0,0,2,2,7,2015
4,Apr 3,SAC,L 115-106,Reg,32:14,2,6,33.3,0,1,...,4,9,13,5,0,1,2,2,6,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,Nov 2,@MIN,L 89-110,Reg,29:58,11,23,47.8,1,6,...,2,8,10,3,5,0,0,0,25,2023
616,Oct 31,UTA,W 102-110,Reg,35:14,12,16,75.0,1,3,...,1,9,10,11,1,0,2,3,27,2023
617,Oct 29,@OKC,W 128-95,Reg,29:52,12,16,75.0,1,2,...,2,12,14,5,4,0,0,3,28,2023
618,Oct 27,@MEM,W 108-104,Reg,34:58,8,14,57.1,1,4,...,3,9,12,7,9,1,1,1,22,2023


# Data wrangling

In [1892]:
df_stats_jokic_raw.isnull().sum()

Date         0
Opp          0
Score        0
Type         0
Min          0
FGM          0
FGA          0
FG%          0
3PM          0
3PA          0
3P%          0
FTM          0
FTA          0
FT%          0
OR           0
DR           0
Reb          0
Ast          0
TO           0
Stl          0
Blk          0
PF           0
Pts          0
Temporada    0
dtype: int64

In [1893]:
df_stats_jokic_raw.columns

Index(['Date', 'Opp', 'Score', 'Type', 'Min', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OR', 'DR', 'Reb', 'Ast', 'TO',
       'Stl', 'Blk', 'PF', 'Pts', 'Temporada'],
      dtype='object')

In [1894]:
df_stats_jokic_raw.head(1)

,Date,Opp,Score,Type,Min,FGM,FGA,FG%,3PM,3PA,...,OR,DR,Reb,Ast,TO,Stl,Blk,PF,Pts,Temporada
0,Apr 14,@POR,L 99-107,Reg,34:56,7,10,70.0,1,2,...,6,7,13,5,2,0,0,3,18,2015


In [1895]:
colunas = {'Date'       : 'data',
           'Opp'        : 'adversario',
           'Score'      : 'placar',
           'Type'       : 'tipo_temporada',
           'Min'        : 'minutos',
           'FGM'        : 'fgm',  # Field goals made
           'FGA'        : 'fga',  # Field goals attempted
           'FG%'        : 'fg_percent',
           '3PM'        : '3pm',  # 3 points made
           '3PA'        : '3pa',  # 3 points attempted
           '3P%'        : '3p_percent',
           'FTM'        : 'ftm',  # Free throws made
           'FTA'        : 'fta',  # Free throws attempted
           'FT%'        : 'ft_percent',
           'OR'         : 'rebote_ataque',
           'DR'         : 'rebote_defesa',
           'Reb'        : 'rebotes',
           'Ast'        : 'assistencias',
           'TO'         : 'turnover',  # Perdas de bola
           'Stl'        : 'roubos',
           'Blk'        : 'bloqueios',
           'PF'         : 'faltas',
           'Pts'        : 'pontos',
           'Temporada'  : 'temporada'}

df_stats_jokic_raw.rename(columns=colunas,
                          inplace=True)

df_stats_jokic = df_stats_jokic_raw.copy()

In [1896]:
df_stats_jokic['mes'] = df_stats_jokic['data'].str.extract(r'([a-zA-Z]+)')

df_stats_jokic['dia'] = df_stats_jokic['data'].str.extract(r'(\d+)').astype(int)

df_stats_jokic['resultado'] = df_stats_jokic['placar'].str.extract(r'([a-zA-Z]+)')

df_stats_jokic['resultado'] = df_stats_jokic['resultado'].map({'L': 'derrota',
                                                               'W': 'vitoria'})

df_stats_jokic['placar'] = df_stats_jokic['placar'].str.extract(r'[A-Za-z]\s*(.*)')

df_stats_jokic['minutos'] = df_stats_jokic['minutos'].str.extract(r'(\d+)').astype(int)


for i, row in df_stats_jokic.iterrows():
    if '@' in row['adversario']:
        df_stats_jokic.loc[i, 'local_jogo'] = 'fora'
        df_stats_jokic.loc[i, 'adversario'] = re.sub(r'@', '', df_stats_jokic['adversario'][i])
    else:
        df_stats_jokic.loc[i, 'local_jogo'] = 'casa'


df_stats_jokic

,data,adversario,placar,tipo_temporada,minutos,fgm,fga,fg_percent,3pm,3pa,...,turnover,roubos,bloqueios,faltas,pontos,temporada,mes,dia,resultado,local_jogo
0,Apr 14,POR,99-107,Reg,34,7,10,70.0,1,2,...,2,0,0,3,18,2015,Apr,14,derrota,fora
1,Apr 10,UTA,100-84,Reg,36,7,9,77.8,0,2,...,1,2,1,4,19,2015,Apr,10,derrota,casa
2,Apr 9,SA,98-102,Reg,32,3,9,33.3,0,2,...,0,1,1,2,8,2015,Apr,9,vitoria,casa
3,Apr 6,OKC,124-102,Reg,19,3,5,60.0,1,3,...,0,0,2,2,7,2015,Apr,6,derrota,casa
4,Apr 3,SAC,115-106,Reg,32,2,6,33.3,0,1,...,0,1,2,2,6,2015,Apr,3,derrota,casa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,Nov 2,MIN,89-110,Reg,29,11,23,47.8,1,6,...,5,0,0,0,25,2023,Nov,2,derrota,fora
616,Oct 31,UTA,102-110,Reg,35,12,16,75.0,1,3,...,1,0,2,3,27,2023,Oct,31,vitoria,casa
617,Oct 29,OKC,128-95,Reg,29,12,16,75.0,1,2,...,4,0,0,3,28,2023,Oct,29,vitoria,fora
618,Oct 27,MEM,108-104,Reg,34,8,14,57.1,1,4,...,9,1,1,1,22,2023,Oct,27,vitoria,fora


In [1897]:
# Regex para separar os placares
df_stats_jokic['placar_1'] = df_stats_jokic['placar'].str.extract(r'(\d+)-').astype(int)
df_stats_jokic['placar_2'] = df_stats_jokic['placar'].str.extract(r'-(\d+)').astype(int)

In [1898]:
# Atribui o placar ao Denver e ao adversário conforme quem ganhou e perdeu
for index, row in df_stats_jokic.iterrows():
    if 'derrota' in df_stats_jokic.loc[index, 'resultado']:
        if df_stats_jokic.loc[index, 'placar_1'] < df_stats_jokic.loc[index, 'placar_2']:
            df_stats_jokic.loc[index, 'placar_denver'] = df_stats_jokic.loc[index, 'placar_1']
            df_stats_jokic.loc[index, 'placar_adversario'] = df_stats_jokic.loc[index, 'placar_2']

        else:
            df_stats_jokic.loc[index, 'placar_denver'] = df_stats_jokic.loc[index, 'placar_2']
            df_stats_jokic.loc[index, 'placar_adversario'] = df_stats_jokic.loc[index, 'placar_1']


    if 'vitoria' in df_stats_jokic.loc[index, 'resultado']:
        if df_stats_jokic.loc[index, 'placar_1'] < df_stats_jokic.loc[index, 'placar_2']:
            df_stats_jokic.loc[index, 'placar_denver'] = df_stats_jokic.loc[index, 'placar_2']
            df_stats_jokic.loc[index, 'placar_adversario'] = df_stats_jokic.loc[index, 'placar_1']

        else:
            df_stats_jokic.loc[index, 'placar_denver'] = df_stats_jokic.loc[index, 'placar_1']
            df_stats_jokic.loc[index, 'placar_adversario'] = df_stats_jokic.loc[index, 'placar_2']

df_stats_jokic.drop(columns=['data', 'placar', 'placar_1', 'placar_2'], inplace=True)

In [1899]:
# Substitui string '-' de colunas numéricas por 0
for i, row in df_stats_jokic.iterrows():
    if '-' in df_stats_jokic.loc[i, 'fg_percent']:
        df_stats_jokic.loc[i, 'fg_percent'] = 0

    if '-' in df_stats_jokic.loc[i, '3p_percent']:
        df_stats_jokic.loc[i, '3p_percent'] = 0

    if '-' in df_stats_jokic.loc[i, 'ft_percent']:
        df_stats_jokic.loc[i, 'ft_percent'] = 0

In [1900]:
df_stats_jokic['fg_percent'] = df_stats_jokic['fg_percent'].astype(float).astype(int)
df_stats_jokic['3p_percent'] = df_stats_jokic['3p_percent'].astype(float).astype(int)
df_stats_jokic['ft_percent'] = df_stats_jokic['ft_percent'].astype(float).astype(int)
df_stats_jokic['placar_denver'] = df_stats_jokic['placar_denver'].astype(int)
df_stats_jokic['placar_adversario'] = df_stats_jokic['placar_adversario'].astype(int)

In [1901]:
df_stats_jokic[['fg_percent', '3p_percent', 'ft_percent', 'placar_denver', 'placar_adversario']].dtypes

fg_percent           int64
3p_percent           int64
ft_percent           int64
placar_denver        int64
placar_adversario    int64
dtype: object

## Identificando as conferências dos times

In [1902]:
df_stats_jokic['adversario'].unique()

array(['POR', 'UTA', 'SA', 'OKC', 'SAC', 'NO', 'MEM', 'DAL', 'LAC', 'LAL',
       'PHI', 'CLE', 'CHA', 'ATL', 'ORL', 'MIA', 'WAS', 'PHO', 'NY',
       'BKN', 'BOS', 'DET', 'CHI', 'TOR', 'IND', 'GS', 'MIN', 'HOU',
       'MIL', 'EAS'], dtype=object)

In [1903]:
index_all_star = df_stats_jokic[df_stats_jokic['adversario'] == 'EAS'].index.values[0]

index_all_star

545

In [1904]:
df_stats_jokic.drop(index=index_all_star, inplace=True)

df_stats_jokic.reset_index(drop=True, inplace=True)

In [1905]:
df_stats_jokic[df_stats_jokic['adversario'] == 'EAS']

,adversario,tipo_temporada,minutos,fgm,fga,fg_percent,3pm,3pa,3p_percent,ftm,...,bloqueios,faltas,pontos,temporada,mes,dia,resultado,local_jogo,placar_denver,placar_adversario


In [1906]:
conferencia_leste = ['ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND', 'MIA', 'MIL', 'NY', 'ORL', 'PHI', 'TOR', 'WSH']
conferencia_oeste = ['DAL', 'DEN', 'GS', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN', 'NO', 'OKC', 'PHX', 'POR', 'SAC', 'SA', 'UTAH']

In [1907]:
def conferencia(time):
    """
    Identifica a qual liga o time pertence
    """
    if time in conferencia_leste:
        return 'leste'
    else:
        return 'oeste'

In [1908]:
df_stats_jokic['conferencia'] = df_stats_jokic['adversario'].apply(conferencia)

In [1909]:
df_stats_jokic.dtypes

adversario           object
tipo_temporada       object
minutos               int64
fgm                   int64
fga                   int64
fg_percent            int64
3pm                   int64
3pa                   int64
3p_percent            int64
ftm                   int64
fta                   int64
ft_percent            int64
rebote_ataque         int64
rebote_defesa         int64
rebotes               int64
assistencias          int64
turnover              int64
roubos                int64
bloqueios             int64
faltas                int64
pontos                int64
temporada             int64
mes                  object
dia                   int64
resultado            object
local_jogo           object
placar_denver         int64
placar_adversario     int64
conferencia          object
dtype: object

## Salva dataset ajustado

In [1910]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/novo_nba_stats_jokic_ajustado.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset from csv

In [1911]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/novo_nba_stats_jokic_ajustado.csv',
                             encoding='utf-8',
                             sep=',')

df_stats_jokic

,adversario,tipo_temporada,minutos,fgm,fga,fg_percent,3pm,3pa,3p_percent,ftm,...,faltas,pontos,temporada,mes,dia,resultado,local_jogo,placar_denver,placar_adversario,conferencia
0,POR,Reg,34,7,10,70,1,2,50,3,...,3,18,2015,Apr,14,derrota,fora,99,107,oeste
1,UTA,Reg,36,7,9,77,0,2,0,5,...,4,19,2015,Apr,10,derrota,casa,84,100,oeste
2,SA,Reg,32,3,9,33,0,2,0,2,...,2,8,2015,Apr,9,vitoria,casa,102,98,oeste
3,OKC,Reg,19,3,5,60,1,3,33,0,...,2,7,2015,Apr,6,derrota,casa,102,124,oeste
4,SAC,Reg,32,2,6,33,0,1,0,2,...,2,6,2015,Apr,3,derrota,casa,106,115,oeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,MIN,Reg,29,11,23,47,1,6,16,2,...,0,25,2023,Nov,2,derrota,fora,89,110,oeste
615,UTA,Reg,35,12,16,75,1,3,33,2,...,3,27,2023,Oct,31,vitoria,casa,110,102,oeste
616,OKC,Reg,29,12,16,75,1,2,50,3,...,3,28,2023,Oct,29,vitoria,fora,128,95,oeste
617,MEM,Reg,34,8,14,57,1,4,25,5,...,1,22,2023,Oct,27,vitoria,fora,108,104,oeste


# Análise probabilística

In [1912]:
analise_jogos = []

for adversario in df_stats_jokic['adversario'].unique():
    quantidade_jogos = len(df_stats_jokic[df_stats_jokic['adversario'] == adversario])

    quantidade_vitorias_denver = len(df_stats_jokic[(df_stats_jokic['adversario'] == adversario) &
                                                    (df_stats_jokic['resultado'] == 'vitoria')])

    prob_vitoria = round(quantidade_vitorias_denver / quantidade_jogos, 2)

    quantidade_derrotas = len(df_stats_jokic[(df_stats_jokic['adversario'] == adversario) &
                                             (df_stats_jokic['resultado'] == 'derrota')])

    prob_derrota = round(quantidade_derrotas / quantidade_jogos, 2)


    analise_jogos.append([adversario, quantidade_jogos,
                          quantidade_vitorias_denver, prob_vitoria,
                          quantidade_derrotas, prob_derrota])

In [1913]:
colunas = ['adversario', 'total_jogos',
           'vitorias', 'prob_vitorias',
           'derrotas', 'prob_derrotas']

df_analise_jogos = pd.DataFrame(data=analise_jogos,
                                columns=colunas)

df_analise_jogos.sort_values(by='prob_vitorias', ascending=False, inplace=True)

df_analise_jogos.reset_index(drop=True, inplace=True)

df_analise_jogos

,adversario,total_jogos,vitorias,prob_vitorias,derrotas,prob_derrotas
0,NY,15,13,0.87,2,0.13
1,CHI,14,11,0.79,3,0.21
2,IND,13,10,0.77,3,0.23
3,PHO,25,19,0.76,6,0.24
4,MIL,14,10,0.71,4,0.29
5,CHA,16,11,0.69,5,0.31
6,DET,16,11,0.69,5,0.31
7,MIA,16,11,0.69,5,0.31
8,ORL,16,11,0.69,5,0.31
9,POR,29,19,0.66,10,0.34


In [1914]:
fig = go.Figure(go.Bar(y=df_analise_jogos['adversario'],
                       x=df_analise_jogos['vitorias'],
                       name='Vitórias',
                       orientation='h',
                       text=df_analise_jogos['vitorias'],
                       textfont=dict(size=12,
                                     color='white')))


fig.add_trace(go.Bar(y=df_analise_jogos['adversario'],
                     x=df_analise_jogos['derrotas'],
                     name='Derrotas',
                     orientation='h',
                     text=df_analise_jogos['derrotas'],
                     textfont=dict(size=12,
                                   color='white')))


fig.update_layout(height=1000,
                  width=1200,
                  showlegend=True,
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50),
                  barmode='stack',
                  yaxis={'categoryorder':'total ascending'})

In [1915]:
bar_widths = np.full(len(df_analise_jogos['prob_vitorias']), 0.4)

fig = go.Figure(go.Bar(x=df_analise_jogos['adversario'],
                       y=df_analise_jogos['prob_vitorias'],
                       name='Probabilidade de vitória',
                       text=df_analise_jogos['prob_vitorias'],
                       textfont=dict(size=12,
                                     color='black'),
                       width=bar_widths))

fig.update_traces(textposition='outside')


fig.add_trace(go.Bar(x=df_analise_jogos['adversario'],
                     y=df_analise_jogos['prob_derrotas'],
                     name='Probabilidade de derrota',
                     text=df_analise_jogos['prob_derrotas'],
                     textfont=dict(size=12,
                                   color='black'),
                     width=bar_widths))

fig.update_traces(textposition='outside')


fig.update_layout(height=600,
                  width=2000,
                  showlegend=True,
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

# Análise descritiva

In [1916]:
df_stats_jokic.columns

Index(['adversario', 'tipo_temporada', 'minutos', 'fgm', 'fga', 'fg_percent',
       '3pm', '3pa', '3p_percent', 'ftm', 'fta', 'ft_percent', 'rebote_ataque',
       'rebote_defesa', 'rebotes', 'assistencias', 'turnover', 'roubos',
       'bloqueios', 'faltas', 'pontos', 'temporada', 'mes', 'dia', 'resultado',
       'local_jogo', 'placar_denver', 'placar_adversario', 'conferencia'],
      dtype='object')

In [1917]:
df_stats_jokic[['fgm', 'fga', 'fg_percent']].describe()

,fgm,fga,fg_percent
count,619.000000,619.000000,619.000000
mean,7.886914,14.277868,54.987076
std,3.807692,6.238025,16.413641
min,0.000000,0.000000,0.000000
25%,5.000000,10.000000,45.000000
50%,8.000000,14.000000,56.000000
75%,10.000000,18.000000,64.000000
max,20.000000,35.000000,100.000000


In [1918]:
df_stats_jokic[['3pm', '3pa', '3p_percent']].describe()

,3pm,3pa,3p_percent
count,619.000000,619.000000,619.000000
mean,1.004847,2.899838,30.006462
std,1.087577,2.230195,31.665474
min,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000
50%,1.000000,2.000000,25.000000
75%,2.000000,4.000000,50.000000
max,6.000000,11.000000,100.000000


In [1919]:
df_stats_jokic[['ftm', 'fta', 'ft_percent']].describe()

,ftm,fta,ft_percent
count,619.000000,619.000000,619.000000
mean,3.743134,4.531502,70.953150
std,2.967504,3.466935,34.465818
min,0.000000,0.000000,0.000000
25%,1.000000,2.000000,60.000000
50%,3.000000,4.000000,83.000000
75%,6.000000,7.000000,100.000000
max,16.000000,18.000000,100.000000


In [1920]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['fgm'], name='De quadra'))

fig.add_trace(go.Box(x=df_stats_jokic['ftm'], name='Lances livres'))

fig.add_trace(go.Box(x=df_stats_jokic['3pm'], name='3 pontos'))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=False,
                  title='Pontos feitos',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

In [1921]:
df_stats_jokic[['pontos', 'placar_denver', 'placar_adversario']].describe()

,pontos,placar_denver,placar_adversario
count,619.000000,619.000000,619.000000
mean,20.521809,111.352181,109.140549
std,9.594400,12.868719,11.623042
min,0.000000,74.000000,74.000000
25%,14.000000,103.000000,101.000000
50%,21.000000,111.000000,109.000000
75%,27.000000,121.000000,117.000000
max,50.000000,146.000000,149.000000


In [1922]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['pontos'], name=''))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=False,
                  title='Total de pontos',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

In [1923]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['placar_denver'], name='Denver'))

fig.add_trace(go.Box(x=df_stats_jokic['placar_adversario'], name='Adversários'))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=False,
                  title='Placar',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

In [1924]:
fig = px.box(df_stats_jokic, x='placar_adversario', y='adversario', color='adversario')

fig.update_layout(height=1500,
                  width=1200,
                  showlegend=False,
                  title='Placar por adversário',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50),
                  xaxis_title='Placar',
                  yaxis_title=None)

fig.update_yaxes(categoryorder='category descending')

In [1925]:
df_stats_jokic[['rebote_ataque', 'rebote_defesa', 'rebotes',
                'assistencias', 'turnover', 'roubos',
                'bloqueios', 'faltas', 'minutos']].describe()

,rebote_ataque,rebote_defesa,rebotes,assistencias,turnover,roubos,bloqueios,faltas,minutos
count,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000
mean,2.647819,7.961228,10.609047,6.751212,2.867528,1.198708,0.715670,2.749596,30.353796
std,1.817403,3.587345,4.215741,3.650528,1.933832,1.116628,0.873891,1.385423,7.003180
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,6.000000,8.000000,4.000000,1.000000,0.000000,0.000000,2.000000,27.000000
50%,2.000000,8.000000,10.000000,6.000000,3.000000,1.000000,0.000000,3.000000,32.000000
75%,4.000000,10.000000,13.000000,10.000000,4.000000,2.000000,1.000000,4.000000,35.000000
max,9.000000,20.000000,27.000000,18.000000,10.000000,7.000000,5.000000,6.000000,47.000000


In [1926]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['rebote_ataque'], name='De ataque'))

fig.add_trace(go.Box(x=df_stats_jokic['rebote_defesa'], name='De defesa'))

fig.add_trace(go.Box(x=df_stats_jokic['rebotes'], name='Totais'))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=False,
                  title='Rebotes',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

In [1927]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['assistencias'], name='Assistências'))

fig.add_trace(go.Box(x=df_stats_jokic['turnover'], name='Turnovers'))

fig.add_trace(go.Box(x=df_stats_jokic['roubos'], name='Roubos de bola'))

fig.add_trace(go.Box(x=df_stats_jokic['bloqueios'], name='Bloqueios'))

fig.add_trace(go.Box(x=df_stats_jokic['faltas'], name='Faltas'))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=True,
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

In [1928]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_stats_jokic['minutos'], name=''))

fig.update_layout(height=600,
                  width=1200,
                  showlegend=False,
                  title='Minutos em quadra',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

# Análise gráfica

In [1929]:
media_porcent_temporadas = df_stats_jokic[['temporada', 'fg_percent', 'ft_percent', '3p_percent']].groupby('temporada').mean()

In [1930]:
fig = px.line(media_porcent_temporadas,
              x=media_porcent_temporadas.index,
              y=['fg_percent', 'ft_percent', '3p_percent'],
              labels={'variable': ''})

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic %stats',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)
fig.update_yaxes(title_text=None)
fig.show()

## Pontos

In [1931]:
media_pontos_temporadas_por_tipo = df_stats_jokic[['temporada', 'fgm', 'ftm', '3pm']].groupby('temporada').mean()
media_pontos_temporadas = df_stats_jokic[['temporada', 'pontos']].groupby('temporada').mean()

In [1932]:
fig = make_subplots(shared_xaxes=True)

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['fgm'],
                         mode='lines',
                         line=dict(color='#845EC2'),
                         name='FG'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['ftm'],
                         mode='lines',
                         line=dict(color='#008BC9'),
                         name='FT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['3pm'],
                         mode='lines',
                         line=dict(color='#FF8066'),
                         name='3PT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas.index,
                         y=media_pontos_temporadas['pontos'],
                         mode='lines',
                         line=dict(color='#936C00'),
                         name='Média geral'))

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic - média de pontos nas temporadas',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)

fig.update_yaxes(title_text=None)

fig.show()

## Correlação

In [1933]:
df_stats_jokic = df_stats_jokic.reindex(sorted(df_stats_jokic.columns), axis=1)

df_stats_jokic

,3p_percent,3pa,3pm,adversario,assistencias,bloqueios,conferencia,dia,faltas,fg_percent,...,placar_denver,pontos,rebote_ataque,rebote_defesa,rebotes,resultado,roubos,temporada,tipo_temporada,turnover
0,50,2,1,POR,5,0,oeste,14,3,70,...,99,18,6,7,13,derrota,0,2015,Reg,2
1,0,2,0,UTA,4,1,oeste,10,4,77,...,84,19,3,8,11,derrota,2,2015,Reg,1
2,0,2,0,SA,4,1,oeste,9,2,33,...,102,8,3,12,15,vitoria,1,2015,Reg,0
3,33,3,1,OKC,2,2,oeste,6,2,60,...,102,7,0,2,2,derrota,0,2015,Reg,0
4,0,1,0,SAC,5,2,oeste,3,2,33,...,106,6,4,9,13,derrota,1,2015,Reg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,16,6,1,MIN,3,0,oeste,2,0,47,...,89,25,2,8,10,derrota,0,2023,Reg,5
615,33,3,1,UTA,11,2,oeste,31,3,75,...,110,27,1,9,10,vitoria,0,2023,Reg,1
616,50,2,1,OKC,5,0,oeste,29,3,75,...,128,28,2,12,14,vitoria,0,2023,Reg,4
617,25,4,1,MEM,7,1,oeste,27,1,57,...,108,22,3,9,12,vitoria,1,2023,Reg,9


In [1934]:
correlacao = df_stats_jokic.drop(['adversario',
                                 'tipo_temporada',
                                 'mes',
                                 'resultado',
                                 'local_jogo',
                                 'conferencia'],
                                 axis=1).corr().round(2)

fig = px.imshow(correlacao, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.update_xaxes(side='top')

fig.show()

±0.9 → correlação muito forte

±0.7 a ±0.9 → correlação forte

±0.5 a ±0.7 → correlação moderada

±0.3 a ±0.5 → correlação fraca

### Correlação com pontos

#### Correlação muito forte

In [1935]:
correlacao.loc[correlacao['pontos'] >= 0.9].index

Index(['fgm', 'pontos'], dtype='object')

#### Correlação forte

In [1936]:
correlacao.loc[(correlacao['pontos'] >= 0.7)
               & (correlacao['pontos'] < 0.9)].index

Index(['fga'], dtype='object')

#### Correlação moderada

In [1937]:
correlacao.loc[(correlacao['pontos'] >= 0.5)
               & (correlacao['pontos'] < 0.7)].index

Index(['fta', 'ftm', 'minutos', 'temporada'], dtype='object')

#### Correlação fraca

In [1938]:
correlacao.loc[(correlacao['pontos'] >= 0.3)
               & (correlacao['pontos'] < 0.5)].index

Index(['3pa', '3pm', 'assistencias', 'fg_percent', 'ft_percent',
       'placar_denver', 'rebote_ataque', 'rebote_defesa', 'rebotes'],
      dtype='object')

In [1939]:
media_acertos_fg = df_stats_jokic['fgm'].mean().round(2)
media_acertos_ft = df_stats_jokic['ftm'].mean().round(2)
media_acertos_3pt = df_stats_jokic['3pm'].mean().round(2)

print('Média de acertos FG por jogo:', media_acertos_fg)
print('Média de acertos FT por jogo:', media_acertos_ft)
print('Média de acertos 3PT por jogo:', media_acertos_3pt)

Média de acertos FG por jogo: 7.89
Média de acertos FT por jogo: 3.74
Média de acertos 3PT por jogo: 1.0


#### Aproveitamento

In [1940]:
media_aproveitamento_porcent_fg = (df_stats_jokic['fg_percent'].mean()).round(2)
media_aproveitamento_porcent_ft = (df_stats_jokic['ft_percent'].mean()).round(2)
media_aproveitamento_porcent_3pt = (df_stats_jokic['3p_percent'].mean()).round(2)

print('Média de aproveitamento FG por jogo:', media_aproveitamento_porcent_fg, '%')
print('Média de aproveitamento FT por jogo:', media_aproveitamento_porcent_ft, '%')
print('Média de aproveitamento 3PT por jogo:', media_aproveitamento_porcent_3pt, '%')

Média de aproveitamento FG por jogo: 54.99 %
Média de aproveitamento FT por jogo: 70.95 %
Média de aproveitamento 3PT por jogo: 30.01 %


# Variáveis categóricas em numéricas

In [1941]:
df_stats_jokic.select_dtypes(include=['object'])

,adversario,conferencia,local_jogo,mes,resultado,tipo_temporada
0,POR,oeste,fora,Apr,derrota,Reg
1,UTA,oeste,casa,Apr,derrota,Reg
2,SA,oeste,casa,Apr,vitoria,Reg
3,OKC,oeste,casa,Apr,derrota,Reg
4,SAC,oeste,casa,Apr,derrota,Reg
...,...,...,...,...,...,...
614,MIN,oeste,fora,Nov,derrota,Reg
615,UTA,oeste,casa,Oct,vitoria,Reg
616,OKC,oeste,fora,Oct,vitoria,Reg
617,MEM,oeste,fora,Oct,vitoria,Reg


In [1942]:
df_dummies = pd.get_dummies(df_stats_jokic[['adversario',
                                          'conferencia',
                                          'local_jogo',
                                          'tipo_temporada',
                                          'mes',
                                          'resultado']],
                            prefix=['adversario',
                                    'conferencia',
                                    'local_jogo',
                                    'tipo_temporada',
                                    'mes',
                                    'resultado'])

df_dummies

,adversario_ATL,adversario_BKN,adversario_BOS,adversario_CHA,adversario_CHI,adversario_CLE,adversario_DAL,adversario_DET,adversario_GS,adversario_HOU,...,mes_Aug,mes_Dec,mes_Feb,mes_Jan,mes_Mar,mes_May,mes_Nov,mes_Oct,resultado_derrota,resultado_vitoria
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [1943]:
df_stats_jokic_var_continuas = df_stats_jokic.drop(columns=df_stats_jokic.select_dtypes(include=['object']).columns)

df_stats_jokic_var_continuas

,3p_percent,3pa,3pm,assistencias,bloqueios,dia,faltas,fg_percent,fga,fgm,...,minutos,placar_adversario,placar_denver,pontos,rebote_ataque,rebote_defesa,rebotes,roubos,temporada,turnover
0,50,2,1,5,0,14,3,70,10,7,...,34,107,99,18,6,7,13,0,2015,2
1,0,2,0,4,1,10,4,77,9,7,...,36,100,84,19,3,8,11,2,2015,1
2,0,2,0,4,1,9,2,33,9,3,...,32,98,102,8,3,12,15,1,2015,0
3,33,3,1,2,2,6,2,60,5,3,...,19,124,102,7,0,2,2,0,2015,0
4,0,1,0,5,2,3,2,33,6,2,...,32,115,106,6,4,9,13,1,2015,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,16,6,1,3,0,2,0,47,23,11,...,29,110,89,25,2,8,10,0,2023,5
615,33,3,1,11,2,31,3,75,16,12,...,35,102,110,27,1,9,10,0,2023,1
616,50,2,1,5,0,29,3,75,16,12,...,29,95,128,28,2,12,14,0,2023,4
617,25,4,1,7,1,27,1,57,14,8,...,34,104,108,22,3,9,12,1,2023,9


In [1944]:
df_stats_jokic_var_continuas = pd.concat([df_stats_jokic_var_continuas, df_dummies], axis=1)

df_stats_jokic_var_continuas = df_stats_jokic_var_continuas.reindex(sorted(df_stats_jokic_var_continuas.columns),
                                                                    axis=1)

df_stats_jokic_var_continuas

,3p_percent,3pa,3pm,adversario_ATL,adversario_BKN,adversario_BOS,adversario_CHA,adversario_CHI,adversario_CLE,adversario_DAL,...,pontos,rebote_ataque,rebote_defesa,rebotes,resultado_derrota,resultado_vitoria,roubos,temporada,tipo_temporada_Reg,turnover
0,50,2,1,0,0,0,0,0,0,0,...,18,6,7,13,1,0,0,2015,1,2
1,0,2,0,0,0,0,0,0,0,0,...,19,3,8,11,1,0,2,2015,1,1
2,0,2,0,0,0,0,0,0,0,0,...,8,3,12,15,0,1,1,2015,1,0
3,33,3,1,0,0,0,0,0,0,0,...,7,0,2,2,1,0,0,2015,1,0
4,0,1,0,0,0,0,0,0,0,0,...,6,4,9,13,1,0,1,2015,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,16,6,1,0,0,0,0,0,0,0,...,25,2,8,10,1,0,0,2023,1,5
615,33,3,1,0,0,0,0,0,0,0,...,27,1,9,10,0,1,0,2023,1,1
616,50,2,1,0,0,0,0,0,0,0,...,28,2,12,14,0,1,0,2023,1,4
617,25,4,1,0,0,0,0,0,0,0,...,22,3,9,12,0,1,1,2023,1,9


In [1945]:
colunas_lower_case = [x.lower() for x in df_stats_jokic_var_continuas.columns.values]

df_stats_jokic_var_continuas = df_stats_jokic_var_continuas.set_axis(colunas_lower_case, axis=1, copy=True)

# Normalização

In [1946]:
scaler = StandardScaler()

df_stats_jokic_scaled = scaler.fit_transform(df_stats_jokic_var_continuas)

df_stats_jokic_scaled = pd.DataFrame(df_stats_jokic_scaled, columns=df_stats_jokic_var_continuas.columns)

df_stats_jokic_scaled

,3p_percent,3pa,3pm,adversario_atl,adversario_bkn,adversario_bos,adversario_cha,adversario_chi,adversario_cle,adversario_dal,...,pontos,rebote_ataque,rebote_defesa,rebotes,resultado_derrota,resultado_vitoria,roubos,temporada,tipo_temporada_reg,turnover
0,0.631909,-0.403806,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,-0.263054,1.845982,-0.268166,0.567608,1.198753,-1.198753,-1.074374,-1.496792,0.0,-0.448969
1,-0.948375,-0.403806,-0.924678,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,-0.158743,0.193939,0.010817,0.092812,1.198753,-1.198753,0.718180,-1.496792,0.0,-0.966495
2,-0.948375,-0.403806,-0.924678,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,-1.306172,0.193939,1.126749,1.042404,-0.834200,0.834200,-0.178097,-1.496792,0.0,-1.484021
3,0.094613,0.044948,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,-1.410484,-1.458103,-1.663082,-2.043771,1.198753,-1.198753,-1.074374,-1.496792,0.0,-1.484021
4,-0.948375,-0.852560,-0.924678,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,-1.514795,0.744620,0.289800,0.567608,1.198753,-1.198753,-0.178097,-1.496792,0.0,-1.484021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,-0.442684,1.391210,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,0.467128,-0.356742,0.010817,-0.144587,1.198753,-1.198753,-1.074374,1.832999,0.0,1.103610
615,0.094613,0.044948,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,0.675751,-0.907422,0.289800,-0.144587,-0.834200,0.834200,-1.074374,1.832999,0.0,-0.966495
616,0.631909,-0.403806,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,0.780063,-0.356742,1.126749,0.805006,-0.834200,0.834200,-1.074374,1.832999,0.0,0.586084
617,-0.158233,0.493702,-0.004460,-0.168045,-0.162893,-0.157589,-0.162893,-0.15212,-0.168045,-0.209392,...,0.154193,0.193939,0.289800,0.330210,-0.834200,0.834200,-0.178097,1.832999,0.0,3.173716


# Principal Component Analysis (PCA)

## Teste de Esfericidade de Bartlett

Hipótese Nula que a matriz de correlação ρ da sua base de dados é estatisticamente igual a uma matriz identidade.

*   Significa que temos uma base de dados onde não existe correlação entre nenhuma das variáveis que compõem essa base. Importante salientar que o teste verifica se as correlações entre as variáveis são ESTATISTICAMENTE IGUAIS a 0, o que não quer dizer que elas sejam 0.


Hipótese Alternativa, temos que a matriz de correlações da sua base de dados não é ESTATISTICAMENTE IGUAL a uma matriz identidade.

Para **rejeitarmos** a Hipótese Nula, que é o que queremos, temos de ter que o valor calculado na distribuição qui quadrada acima deve ser maior que o valor dessa mesma distribuição para o nível de significância desejado. Se for maior, podemos rejeitar a hipótese nula e aceitarmos a hipótese alternativa.

Fonte: [Abrindo a Caixa Preta: PCA (Análise de Componentes Principais)](https://medium.com/@felipeverasaraujo/abrindo-a-caixa-preta-pca-an%C3%A1lise-de-componentes-principais-d5d400781dfe)

In [1947]:
# for i, coluna in enumerate(df_stats_jokic_scaled.columns):
#     coluna_index = "df_stats_jokic_scaled['" + str(coluna) + "']"
#     print(coluna_index)

In [1948]:
# stats, p_value = bartlett(df_stats_jokic_scaled['3p_percent'],
#                           df_stats_jokic_scaled['3pa'],
#                           df_stats_jokic_scaled['3pm'],
#                           df_stats_jokic_scaled['adversario_atl'],
#                           df_stats_jokic_scaled['adversario_bkn'],
#                           df_stats_jokic_scaled['adversario_bos'],
#                           df_stats_jokic_scaled['adversario_cha'],
#                           df_stats_jokic_scaled['adversario_chi'],
#                           df_stats_jokic_scaled['adversario_cle'],
#                           df_stats_jokic_scaled['adversario_dal'],
#                           df_stats_jokic_scaled['adversario_det'],
#                           df_stats_jokic_scaled['adversario_gs'],
#                           df_stats_jokic_scaled['adversario_hou'],
#                           df_stats_jokic_scaled['adversario_ind'],
#                           df_stats_jokic_scaled['adversario_lac'],
#                           df_stats_jokic_scaled['adversario_lal'],
#                           df_stats_jokic_scaled['adversario_mem'],
#                           df_stats_jokic_scaled['adversario_mia'],
#                           df_stats_jokic_scaled['adversario_mil'],
#                           df_stats_jokic_scaled['adversario_min'],
#                           df_stats_jokic_scaled['adversario_no'],
#                           df_stats_jokic_scaled['adversario_ny'],
#                           df_stats_jokic_scaled['adversario_okc'],
#                           df_stats_jokic_scaled['adversario_orl'],
#                           df_stats_jokic_scaled['adversario_phi'],
#                           df_stats_jokic_scaled['adversario_pho'],
#                           df_stats_jokic_scaled['adversario_por'],
#                           df_stats_jokic_scaled['adversario_sa'],
#                           df_stats_jokic_scaled['adversario_sac'],
#                           df_stats_jokic_scaled['adversario_tor'],
#                           df_stats_jokic_scaled['adversario_uta'],
#                           df_stats_jokic_scaled['adversario_was'],
#                           df_stats_jokic_scaled['assistencias'],
#                           df_stats_jokic_scaled['bloqueios'],
#                           df_stats_jokic_scaled['conferencia_leste'],
#                           df_stats_jokic_scaled['conferencia_oeste'],
#                           df_stats_jokic_scaled['dia'],
#                           df_stats_jokic_scaled['faltas'],
#                           df_stats_jokic_scaled['fg_percent'],
#                           df_stats_jokic_scaled['fga'],
#                           df_stats_jokic_scaled['fgm'],
#                           df_stats_jokic_scaled['ft_percent'],
#                           df_stats_jokic_scaled['fta'],
#                           df_stats_jokic_scaled['ftm'],
#                           df_stats_jokic_scaled['local_jogo_casa'],
#                           df_stats_jokic_scaled['local_jogo_fora'],
#                           df_stats_jokic_scaled['mes_apr'],
#                           df_stats_jokic_scaled['mes_aug'],
#                           df_stats_jokic_scaled['mes_dec'],
#                           df_stats_jokic_scaled['mes_feb'],
#                           df_stats_jokic_scaled['mes_jan'],
#                           df_stats_jokic_scaled['mes_mar'],
#                           df_stats_jokic_scaled['mes_may'],
#                           df_stats_jokic_scaled['mes_nov'],
#                           df_stats_jokic_scaled['mes_oct'],
#                           df_stats_jokic_scaled['minutos'],
#                           df_stats_jokic_scaled['placar_adversario'],
#                           df_stats_jokic_scaled['placar_denver'],
#                           df_stats_jokic_scaled['pontos'],
#                           df_stats_jokic_scaled['rebote_ataque'],
#                           df_stats_jokic_scaled['rebote_defesa'],
#                           df_stats_jokic_scaled['rebotes'],
#                           df_stats_jokic_scaled['resultado_derrota'],
#                           df_stats_jokic_scaled['resultado_vitoria'],
#                           df_stats_jokic_scaled['roubos'],
#                           df_stats_jokic_scaled['temporada'],
#                           df_stats_jokic_scaled['tipo_temporada_reg'],
#                           df_stats_jokic_scaled['turnover'])

In [1949]:
# significancia = 0.05

In [1950]:
# p_value

In [1951]:
# if p_value < significancia:
#     print('Hipótese nula rejeitada\nÉ possível utilizar PCA')
# else:
#     print('Hipótese nula aceita\nNão é possível utilizar PCA')

## PCA

1. Padronização dos dados
2. Obtenção dos autovalores e autovetores
3. Obtenção das componentes principais
4. Correlação entre valores padronizados e scores dos componentes principais

In [1952]:
# pca = PCA(random_state=0)
# pca = pca.fit(df_stats_jokic_scaled)

In [1953]:
# pca.n_components_

In [1954]:
# fig = px.line(x=range(1, len(pca.explained_variance_) + 1),
#               y=pca.explained_variance_,
#               labels={'x': 'N de componentes', 'y': 'Eigenvalues'})

# fig.update_layout(margin=dict(t=50,
#                               r=50,
#                               b=50,
#                               l=50),
#                   showlegend=False,
#                   width=1000,
#                   height=400,
#                   title='Scree plot')

# fig.add_hline(y=1, line_width=2, line_dash='dash', line_color='red')

# fig.show()

* **Autovalores**: Para o universo da PCA, podemos dizer que os autovalores mostram quantas variáveis cada autovalor representa em termos de variância. Quando dividimos os Autovalores pelo número de variáveis na base de dados, conseguimos entender a variância explicada que cada fator terá sobre a variância total das variáveis originais.

* **Autovetores**: Para o universo da PCA, podemos definir os autovetores como sendo as projeções das variâncias capturadas das variáveis originais. Ele vai mostrar o quanto cada fator está capturando da variância de cada variável.

*Obs: Enquanto o autovalor vai mostrar o quanto cada fator explica da variância TOTAL das variáveis, os autovetores vão mostrar o quanto da variância DE CADA variável está sendo explicada por cada fator.*

Fonte: [Abrindo a Caixa Preta: PCA (Análise de Componentes Principais)](https://medium.com/@felipeverasaraujo/abrindo-a-caixa-preta-pca-an%C3%A1lise-de-componentes-principais-d5d400781dfe)

### 2. Obtenção dos autovalores e autovetores

In [1955]:
# matriz_autovalores = np.round(pca.explained_variance_, 2)  # Autovalores
# # Variância explicada por cada uma das componentes principais
# matriz_autovalores

In [1956]:
# (pca.explained_variance_).sum()  # Variância total do conjunto de dados normalizados

In [1957]:
# matriz_autovetores = pca.components_
# matriz_autovetores.shape

In [1958]:
# np.round(pca.explained_variance_ratio_, 2)
# # Cada valor é o resultado da divisão de cada autovalor pela soma dos autovalores

In [1959]:
# exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

# fig = px.area(x=range(1, pca.n_components_ + 1),
#               y=exp_var_cumul,
#               labels={'x': 'Nº de components', 'y': 'Variância explicada'})

# fig.update_layout(yaxis=dict(tickmode='linear',
#                                tick0=0,
#                                dtick=0.1),
#                   xaxis=dict(tickmode='linear',
#                                tick0=0,
#                                dtick=5),
#                   margin=dict(t=50,
#                               r=50,
#                               b=50,
#                               l=50),
#                   showlegend=False,
#                   width=1000,
#                   height=400)

# fig.add_hline(y=0.8, line_width=2, line_dash='dash', line_color='green')

# fig.add_hline(y=0.9, line_width=2, line_dash='dash', line_color='red')

# fig.show()

In [1960]:
# exp_var_cumul_series = pd.Series(exp_var_cumul)

# exp_var_cumul_series_90_porcent = exp_var_cumul_series[exp_var_cumul_series >= 0.9]

# n_components = exp_var_cumul_series_90_porcent.index[0]
# # Número mínimo de componentes principais que explicam 90%
# # da variação do conjunto de dados
# n_components


### Nº de componentes definido

In [1961]:
# nomes_colunas = {}

# for i, componente_principal in enumerate(range(1, n_components + 1)):
#     nome_componente = 'CP' + str(componente_principal)
#     colunas = {i: nome_componente}
#     nomes_colunas.update(colunas)

# nomes_colunas

In [1962]:
# pca_n_components = PCA(random_state=0, n_components=n_components)
# pca_n_components = pca.fit(df_stats_jokic_scaled)

In [1963]:
# matriz_autovetores_n_components = pca_n_components.components_

### 3. Obtenção das componentes principais

In [1964]:
# matriz_autovetores_n_components.shape

In [1965]:
# matriz_autovetores_12_components = matriz_autovetores_n_components[:, 0:12]
# matriz_autovetores_12_components.shape

In [1966]:
# matrix_componentes_principais = np.matmul(df_stats_jokic_scaled, matriz_autovetores_12_components)
# matrix_componentes_principais.rename(columns=nomes_colunas, inplace=True)

### 4. Correlação entre valores padronizados e scores dos componentes principais

In [1967]:
# # Calculate correlations between each column of df_stats_jokic_scaled and matrix_componentes_principais
# correlations = pd.DataFrame(index=df_stats_jokic_scaled.columns, columns=matrix_componentes_principais.columns)

# for col1 in df_stats_jokic_scaled.columns:
#     for col2 in matrix_componentes_principais.columns:
#         correlation_value = np.corrcoef(df_stats_jokic_scaled[col1], matrix_componentes_principais[col2])[0, 1]
#         correlations.loc[col1, col2] = correlation_value.round(2)

# # Convert the correlation values to numeric type
# correlations = correlations.apply(pd.to_numeric)

In [1968]:
# df_correlacoes = pd.DataFrame(correlations)
# df_correlacoes.rename(columns=nomes_colunas, inplace=True)

# fig = px.imshow(df_correlacoes,
#                 text_auto=True,
#                 aspect='auto',
#                 color_continuous_scale='RdBu_r',
#                 title='Correlação entre valores padronizados e scores das componentes principais')

# fig.update_layout(margin=dict(t=80,
#                               r=50,
#                               b=50,
#                               l=50),
#                   showlegend=False,
#                   width=1200,
#                   height=1200)

# fig.update_xaxes(side='top')

# fig.show()